## 1. Basic Format

```python
import cv2

def main():
    cap = cv2.VideoCapture(0) # 초기화

    while True: 
        success, image = cap.read() # 영상 읽기
        if success:
            cv2.imshow(f"Finger Counter", image) # 영상 표시
            if cv2.waitKey(1) & 0xFF == ord("q"): # 종료 조건
                break
        else:
            print(f"cap.read() error")
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
```

## 2. Detail Design

```python
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import logging, coloredlogs

from utils import draw_fingercount_on_image

logger = logging.getLogger(__name__)
coloredlogs.install(level="DEBUG", logger=logger)  # logger 설정


def main():
    cap = cv2.VideoCapture(0)
    # TODO 핑거 카운터 알고리즘 설정
    # TODO 동영상 저장 설정

    # https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python#video
    base_options = python.BaseOptions(model_asset_path="models/hand_landmarker.task")
    options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
    with vision.HandLandmarker.create_from_options(options) as detector:
        while True:
            success, image_origin = cap.read()

            if success:
                image = cv2.cvtColor(image_origin, cv2.COLOR_BGR2RGB)
                rgb_frame = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
                detection_result = detector.detect(rgb_frame)  # notebook/HandLandmarkerResult.ipynb 참조

                if detection_result:  # detection_result 로그데이터 생성
                    logger.debug(f"detection_result:\n {detection_result}")

                annotated_image = draw_fingercount_on_image(  # 원본 카메라 영상에 detection 결과를 합성
                    rgb_image=image_origin,
                    detection_result=detection_result,
                    # TODO 핑거 카운터 알고리즘 추가
                )

                cv2.imshow(f"Finger Counter", annotated_image)  # 동영상 재생
                # TODO 동영상 저장 코드 추가

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
            else:
                logger.critical(f"cap.read() error")
                break

        cap.release()
        cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
```

## 3. Algorithm Design
 - 클래스의 상속기능을 사용할 수 있도록 알고리즘 함수를 클래스로 `Wrapping` 한다
 - 다양한 알고리즘을 그룹화(상속기능 활용)하고 command line 코드 실행시 변경 적용이 가능하다. --algo 옵션
 - 클래스 인스턴스의 `독립적 상태저장` 기능을 활용하여 다양한 `통계정보`를 수집할 수 있다.
 - 독립된 logger 객체를 가지고 있다. command line --log_a arg를 사용하여 알고리즘에 별도 로그 레벨을 지정할 수 있다

```python
class Algorithm:  # algorithm 베이스 클래스, 알고리즘 그룹화에 사용
    def detect_tip_finger(self, hand_landmarks):
        """returns tuple(wrist_point, tip_fingers)"""
        raise NotImplementedError

    def __init__(self):
        from utils import get_args
        self.logger = logging.getLogger(__name__)
        coloredlogs.install(level=get_args().log_a.upper(), logger=self.logger)

    def stats(self):
        pass


class John(Algorithm):
    def detect_tip_finger(self, hand_landmarks):
        ...
        return wrist_point, np.array(tip_fingers)


class Jane(Algorithm):
    def detect_tip_finger(self, hand_landmarks):
       ...
        return wrist_point, np.array(tip_fingers)

                ⁞

```

### 📌  `detect_tip_finger(self, hand_landmarks)` 함수 작성 요령

```python
class Random(Algorithm):
    def detect_tip_finger(self, hand_landmarks) -> tuple(wrist_point, np.array(tip_fingers)): 
        wrist_point = hand_landmarks[0] # 손목(wrist) landmark 데이터
            ...
        tip_fingers = [1, 0, 1, 0, 1] # 5개 손가락 Gesture (1 펴짐, 0 접힘)
        return wrist_point, np.array(tip_fingers)

 - detect_tip_finger 함수는 입력과 출력 포맷이 정해져 있다
 - 함수 작성시 출력 포맷을 따라야 에러가 발생하지 않는다
 - 출력 포맷 : tuple(Landmark, np.array(list[bool, bool, bool, bool, bool]))

## 4. Usage

```bash
❯ python main.py --help
usage: main.py [-h] [--algo {john,jane}] [--log {debug,info,warning,error,critical}] [--log_a {debug,info,warning,error,critical}] [--save]

options:   
  -h, --help            show this help message and exit   
  --algo {john,jane}   
                        select finger counter algorithm   
  --log {debug,info,warning,error,critical}   
                        select log level   
  --log_a {debug,info,warning,error,critical}   
                        select algorithm log level   
  --save                save video data as .avi file   


❯ python main.py --algo john
❯ python main.py --algo jane
❯ python main.py --algo jane --log dubug
❯ python main.py --algo jane --log dubug --save
...
```